In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from time import sleep
import os
from time import perf_counter

########################################################################

driver = webdriver.Firefox()

# driver.get("http://www.python.org")
# assert "Python" in driver.title
# elem = driver.find_element(By.NAME, "q")
# elem.clear()
# elem.send_keys("pycon")
# elem.send_keys(Keys.RETURN)
# assert "No results found." not in driver.page_source
# driver.close()

#---

In [ ]:
login_url   = "https://www.linkedin.com/login/en"
password    = os.getenv('LINKEDIN_PASS')
user        = os.getenv('LINKEDIN_USER')

login(driver    = driver,
      login_url = login_url,
      user      = user,
      password  = password
)

#---

In [16]:
def login(driver, login_url, user, password, timeout_s=10):
    
    # login elements' xpath
    username_xpath = '//*[@id="username"]'
    password_xpath = '//*[@id="password"]'
    
    driver.get(login_url)
    assert "LinkedIn Login" in driver.title
    
    if "Welcome back" not in driver.page_source:
        username_elem = driver.find_element(By.XPATH, username_xpath)
        username_elem.send_keys(user)
    
    password_elem = driver.find_element(By.XPATH, password_xpath)

    password_elem.send_keys(password)
    password_elem.send_keys(Keys.RETURN)
    
    start = perf_counter()
    
    while "Feed" not in driver.title:
        sleep(1)
        
        timer_has_expired = (perf_counter() - start) > timeout_s
        
        if timer_has_expired:
            return False    
    
    return True
        
#---

In [26]:
search_root_url = "https://www.linkedin.com/jobs/search/?"
keywords = "machine learning engineer"
location = "Brazil"
entry_level = True
remote_jobs = True
worldwide = location == "Worldwide"

search_url = search_root_url + \
    ("f_E=2"            if entry_level  else "") + \
    ("&f_WT=2"          if remote_jobs  else "") + \
    ("&geoId=92000000"  if worldwide    else "") + \
    f"&keywords={keywords}" + \
    f"&location={location}" + \
    "&refresh=true"

print(search_url)

driver.get(search_url)

assert f"{keywords} Jobs in {location}" in driver.title

no_of_results_elem = driver.find_element(By.TAG_NAME, "small")

print(no_of_results_elem.text.split()[0])

https://www.linkedin.com/jobs/search/?f_E=2&f_WT=2&keywords=machine learning engineer&location=Brazil&refresh=true
394


### Filters

In [11]:
# EXPERIENCE LEVEL
xp_level = {
    "prop"          : "f_E",
    "internship"    : 1,
    "entry_level"   : 2,
    "associate"     : 3,
    "mid_senior"    : 4,
    "director"      : 5,
    "executive"     : 6,
}

# DATE POSTED
dt_posted = {
    "prop"      : "f_TPR",
    "past_24H"  : "r86400",
    "past_week" : "r604800",
    "past_month": "r2592000",
}

# JOB TYPE f_JT=F
job_type = {
    "prop"          : "f_JT",
    "fulltime"      : "F",
    "part_time"     : "P",
    "contract"      : "C",
    "temporary"     : "T",
    "volunteer"     : "V",
    "internship"    : "I",
    "other"         : "O",
}

work_type = {
    "prop"      : "f_WT",
    "on_site"   : 1,
    "remote"    : 2,
    "hybrid"    : 3,
}

# @todo: map companies code 